### RNA-DNA sample match

In [8]:
import pdb
rna_dict = {}
rna_ids = open('/Users/brownm28/Documents/2019-Jan-31_rna_fusion/run_info/completed/2019-Mar-12_complete_cbttc_RNA_info.txt')
head = next(rna_ids)
r_skip = {}
samp_ids = {}
# catalog rna ids
for line in rna_ids:
    info = line.rstrip('\n').split('\t')
    samp_name = info[6].split('.')[0]
    if info[5] == 'Derived Cell Line':
        samp_name += '-CL'
    bs_id = info[0]
    if samp_name in rna_dict:
        r_skip[samp_name] = 1
    else:
        rna_dict[samp_name] = bs_id
        samp_ids[samp_name] = 0
# catalog dna ids
dna_dict = {}
dna_ids = open('/Users/brownm28/Documents/2019-Jan-30_reharmonization/ngs_checkmate_cbttc/2019-Mar-12_complete_cbttc_DNA_info.txt')
head = next(dna_ids)
d_skip = {}
for line in dna_ids:
    info = line.rstrip('\n').split('\t')
    samp_name = info[6].split('.')[0]
    if samp_name[-1] != 'N':
        if info[5] == 'Derived Cell Line':
            samp_name += '-CL'
        bs_id = info[0]
        if samp_name in dna_dict:
            d_skip[samp_name] = 1
        else:
            dna_dict[samp_name] = bs_id
            samp_ids[samp_name] = 0
# first pass pairs
out = open('/Users/brownm28/Documents/2019-Jan-30_reharmonization/ngs_checkmate_cbttc/dna_rna_pairs.txt', 'w')
out.write('DNA BS ID\tRNA BS ID\tSample name used\tOriginal DNA name\tOriginal RNA name\n')
for samp_name in samp_ids:
    if samp_name not in r_skip and samp_name not in d_skip and samp_name in dna_dict and samp_name in rna_dict:
        out.write(dna_dict[samp_name] + '\t' + rna_dict[samp_name] + '\t' + samp_name + '\t' + samp_name + '\t' + samp_name + '\n')
        samp_ids[samp_name] = 1

d_short = {}
r_short = {}
short_ids = {}
for samp_name in samp_ids:
    if samp_ids[samp_name] == 0:
        samp_split = samp_name.split('-')
        short = samp_name
        if len(samp_split) < 4:
            short = '-'.join(samp_split[0:3])
        if samp_name in dna_ids:
            if short in d_short:
                d_skip[short] = 1
            else:
                d_short[short] = samp_name
                short_ids[short] = 0
        if samp_name in rna_ids:
            if short in r_short:
                r_skip[short] = 1
            else:
                r_short[short] = samp_name
                short_ids[short] = 0
skip = open('/Users/brownm28/Documents/2019-Jan-30_reharmonization/ngs_checkmate_cbttc/skipped.txt', 'w')
skip.write('BS ID\tSample name\tReason\n')
for short in short_ids:
    if short not in d_skip and short not in r_skip and short in d_short and short in r_short:
        short_ids[short] = 1
        samp_ids[dna_ids[d_short[short]]] = 1
        samp_ids[rna_ids[r_short[short]]] = 1
        out.write(dna_dict[d_short[short]] + '\t' + rna_dict[r_short[short]] + '\t' + d_short[short] + '\t' + d_short[short] + '\t' + r_short[short] + '\n')
for samp_name in samp_ids:
    if samp_ids[samp_name] == 0:
        if samp_name in dna_dict:
            skip.write(dna_dict[samp_name] + '\t' + samp_name)
            if samp_name in d_skip:
                skip.write('\tRepeat sample name\n')
            else:
                skip.write('\tNo match found\n')
        if samp_name in rna_dict:
            skip.write(rna_dict[samp_name] + '\t' + samp_name)
            if samp_name in r_skip:
                skip.write('\tRepeat sample name\n')
            else:
                skip.write('\tNo match found\n')
out.close()
skip.close()
